# Ingesting Sentinel-2 L2A data from Microsoft Planetary Computer

## Overview

In this notebook, we will take a look at how to interact with [SpatioTemporal Asset Catalogs (STAC)](https://stacspec.org/en) using [PySTAC](https://pystac.readthedocs.io/en/stable/).

In [ ]:
import os
import pandas as pd
import xarray as xr
import stackstac
import pystac_client
import planetary_computer
import panel as pn
import panel.widgets as pnw
import hvplot.xarray
import holoviews as hv
import geoviews as gv
from pystac.extensions.eo import EOExtension as eo
import datetime
import requests
# import xmltodict
from cartopy import crs
from dask.distributed import Client, LocalCluster

xr.set_options(keep_attrs=True)
hv.extension('bokeh')
gv.extension('bokeh')

In [ ]:
cluster = LocalCluster(n_workers=os.cpu_count())
client = Client(cluster)
client

# Open the catalog

In [ ]:
stac_root = 'https://planetarycomputer.microsoft.com/api/stac/v1'
stac_s2 = 'https://planetarycomputer.microsoft.com/api/stac/v1/collections/sentinel-2-l2a'
catalog = pystac_client.Client.open(
    stac_root,
    modifier=planetary_computer.sign_inplace
)
print(f"{catalog.title} - {catalog.description}")

In [ ]:
sentinel2_collections = [collection for collection in catalog.get_collections() if "sentinel-2" in collection.id]
sentinel2_collections

# Search the collection

In [ ]:
bbox = [-105.283263,39.972809,-105.266569,39.987640] # NCAR, boulder, CO. bbox from http://bboxfinder.com/
dt = "2022-01-01/2023-01-31" # 2022, month of January
collection = "sentinel-2-l2a" # full id of collection
cloud_thresh = 40

In [ ]:
search = catalog.search(
    collections = sentinel2_collections,
    bbox = bbox,
    datetime = dt,
    query={"eo:cloud_cover": {"lt": cloud_thresh}}
)
items = search.item_collection()
print(f"Found {len(items)} items in the {collection}")

Let's get the band names that we are interested in

In [ ]:
first_item = items.items[0]
all_bands = list(first_item.assets.keys())
print(*all_bands, sep=', ')

In [ ]:
bands_of_interest = [b for b in all_bands if b.startswith('B')]

da = stackstac.stack(
    items,
    bounds_latlon=bbox,
    assets=bands_of_interest,
    chunksize='100MiB'
).persist()
da

In [ ]:
# from https://planetarycomputer.microsoft.com/dataset/sentinel-2-l2a#Baseline-Change
def harmonize_to_old(data):  
    """
    Harmonize new Sentinel-2 data to the old baseline.

    Parameters
    ----------
    data: xarray.DataArray
        A DataArray with four dimensions: time, band, y, x

    Returns
    -------
    harmonized: xarray.DataArray
        A DataArray with all values harmonized to the old
        processing baseline.
    """
    cutoff = datetime.datetime(2022, 1, 25)
    offset = 1000
    bands = ["B01","B02","B03","B04","B05","B06","B07","B08","B8A","B09","B10","B11","B12"]

    old = data.sel(time=slice(cutoff))

    to_process = list(set(bands) & set(data.band.data.tolist()))
    new = data.sel(time=slice(cutoff, None)).drop_sel(band=to_process)

    new_harmonized = data.sel(time=slice(cutoff, None), band=to_process).clip(offset)
    new_harmonized -= offset

    new = xr.concat([new, new_harmonized], "band").sel(band=data.band.data.tolist())
    return xr.concat([old, new], dim="time")

da = harmonize_to_old(da)
da

In [ ]:
# Seems like there is duplicate data in the time dimension
da = da.drop_duplicates(dim='time')

In [ ]:
da

In [ ]:
da = (da / da.max(dim='band'))
da.persist()

In [ ]:
# da.sel(band=['B04', 'B03', 'B02']).sel(time='2022-05-30', method='nearest').plot.imshow(robust=True, size=5, aspect=1)  # works

In [ ]:
# da.sel(band=['B04', 'B03', 'B02']).isel(time=0).hvplot(
#     x='x',
#     y='y',
#     geo=True,
#     tiles="ESRI",
#     xlabel="Longitude", ylabel="Latitude",
#     clabel="Surface Reflectance"
# )

In [ ]:
ds = da.to_dataset(dim='band').persist()
ds

In [ ]:
da.sel(band=['B04', 'B03', 'B02'])

In [ ]:
season_names = {
    1: 'Winter',
    2: 'Spring',
    3: 'Summer',
    4: 'Fall'
}


def rgb_during(time):
    da_rgb = da.sel(band=['B04', 'B03', 'B02'])
    start_date = pd.to_datetime(da_rgb['time'].min().data).to_pydatetime()
    end_date = pd.to_datetime(da_rgb['time'].max().data).to_pydatetime()
    closest_date = pd.to_datetime(da_rgb.sel(time=time, method='nearest').time.data).to_pydatetime()
    dt_slider = pnw.DateSlider(name='Date', start=start_date, end=end_date, value=closest_date)
    
    def get_obs_on(t):
        season_key = [month%12 // 3 + 1 for month in range(1, 13)][t.month]
        season = season_names[season_key]
        return da_rgb.sel(time=t).hvplot.rgb(x='x', y='y', bands='band', data_aspect=1, geo=True, tiles='ESRI', rasterize=True, title=f"{season}: {t.strftime('%Y-%m-%d')}")
    
    return pn.panel(pn.Column(
                pn.bind(get_obs_on, t=dt_slider), 
                pn.Row(
                    pn.Spacer(width=60),
                    dt_slider,
                )
            ))
rgb_during(winter)

In [ ]:
winter = '2022-01-15'
spring = '2022-04-15'
summer = '2022-08-01'
fall = '2022-09-15'

winter_plot = rgb_during(winter)
spring_plot = rgb_during(spring)
summer_plot = rgb_during(summer)
fall_plot = rgb_during(fall)

pn.Column(
    pn.Row(winter_plot, spring_plot),
    pn.Row(summer_plot, fall_plot)
)